# Описание хода работы

Импортим нужные библиотеки

In [ ]:
import pandas as pd

Загружаем данные и делаем предобработку

Для начала очищаем пропуски

In [ ]:
# Загрузка данных из CSV-файла
file_path = 'valid.csv' 
data = pd.read_csv(file_path)

# Определяем порог пропусков
threshold = 0.1

# Вычисляем долю пропусков для каждого столбца
missing_fraction = data.isnull().mean()

# Оставляем только те столбцы, где доля пропусков меньше или равна 70%
cleaned_data = data.loc[:, missing_fraction <= threshold]

Затем чистим нули

In [ ]:
# Определяем порог значений
threshold = 0.1

# Создаем DataFrame без столбца 'target' для анализа
data_without_target = data.drop(columns=['target'], errors='ignore')

# Вычисляем долю значений, равных 0 для каждого столбца
zero_fraction = (data_without_target == 0).mean()

# Получаем список столбцов, которые нужно оставить
columns_to_keep = zero_fraction[zero_fraction < threshold].index.tolist()

# Проверяем, есть ли 'target' в исходных данных и добавляем его в список
if 'target' in data.columns:
    columns_to_keep.append('target')

# Оставляем только те столбцы
cleaned_data = data[columns_to_keep]

Чистим не именно пропуски, а null значения


In [ ]:
# Определяем порог пропусков
threshold = 0.1

# Вычисляем долю пропусков для каждой строки
missing_fraction = data.isnull().mean(axis=1)

# Удаляем строки, где доля пропусков больше 10%
cleaned_data = data[missing_fraction <= threshold]

Ощищаем черезмерное количество единичных значений

In [ ]:
def drop_columns_with_high_one_percentage(df, threshold=0.3):
    # Определяем количество строк в DataFrame
    total_rows = df.shape[0]

    # Находим столбцы, которые нужно удалить
    columns_to_drop = []
    for column in df.columns:
        # Вычисляем процент значений равных 1.0
        percentage_of_ones = (df[column] == 1.0).mean()
        if percentage_of_ones > threshold:
            columns_to_drop.append(column)
            print(f"Столбец '{column}' будет удален (процент 1.0: {percentage_of_ones * 100:.2f}%)")

    # Удаляем столбцы
    df.drop(columns=columns_to_drop, inplace=True)
    return df


# Применяем функцию
df_cleaned = drop_columns_with_high_one_percentage(df)

Конвертируем данные во float

In [ ]:
def convert_columns_to_float(df):
    for column in df.columns:
        try:
            # Пробуем преобразовать столбец в float
            df[column] = df[column].astype(float)
        except ValueError:
            # Если возникает ошибка, столбец не может быть преобразован
            print(f"Столбец '{column}' не может быть преобразован в float.")
    return df

# Применяем функцию
df_converted = convert_columns_to_float(df)

Очищаем дубликаты

In [ ]:
def count_matches(row, df):
    return (df == row).sum(axis=1)

# Создаем маску для удаления строк
rows_to_drop = []

for index, row in data.iterrows():
    match_count = count_matches(row, data)
    # Удаляем текущую строку из подсчета совпадений
    match_count = match_count.drop(index)
    if (match_count > 10).any():  # Если есть строка с более чем 10 совпадениями
        rows_to_drop.append(index)

# Удаляем строки из DataFrame
cleaned_data = data.drop(index=rows_to_drop)

Далее отправляем данные в модель

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd

# Загрузка данных
file_path = 'vcleaned_file_without_duplicates.csv'
valid = pd.read_csv(file_path)
file_path = 'cleaned_file_without_duplicates.csv'
train = pd.read_csv(file_path)



# Загрузка Train
X_train = train.drop(columns='target')
y_train = train['target']
X_test = valid.drop(columns='target')
y_test = valid['target']

# инициализация и обучение модели
rf_classifier = RandomForestClassifier(min_samples_leaf=5, n_estimators=100, random_state=384)
rf_classifier.fit(X_train, y_train)

# предсказание вероятностей
y_pred_proba = rf_classifier.predict_proba(X_test)[:,1]

fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)

# построение ROC кривой
plt.plot(fpr, tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('ROC.png')

auc = metrics.roc_auc_score(y_test, y_pred_proba)
print("AUC: %.3f" % auc)

![My Image](ROC.png)

AUC: 0.725
